In [61]:
# import Packages
using CUDA
using Test
using BenchmarkTools
using Random
using LinearAlgebra
using Distributions

Set problem parameters

In [62]:
Base.@kwdef struct Params
    M :: Int64
    N :: Int64
    L :: Int64
    n :: Int64
    T :: Int64
end

Base.@kwdef struct Constraints
    Ulim :: Float64
    α :: Float32
    ϵ :: Float32
    δ :: Float64

    x1_upperlim :: Float64
    x1_lowerlim :: Float64
    y1_upperlim :: Float64
    y1_lowerlim :: Float64

    x2_upperlim :: Float64
    x2_lowerlim :: Float64
    y2_upperlim :: Float64
    y2_lowerlim :: Float64
end

Constraints

Define dynamics

In [63]:
Base.@kwdef mutable struct Model
    f::Function
    h::Function
    u::Function
    Q::Matrix{Float64}
    R::Float64
end

## motion model dynamics
function f(x,u,w)
    xk = Vector{Float64}(undef, 2)
    
    xk[1] = 0.9*x[1] + 0.2*x[2] + w[1]
    xk[2] = -0.15*x[1] + 0.9*x[2] + 0.05*x[1]*x[2] + u + w[2]

    return xk
end

## measurement model dynamics
function h(x,v)
    return x[1,:] .+ v
end

## controller
function u(x)
    return -0.05*x[1]*x[2]
end

u (generic function with 1 method)

In [64]:
#  process noise variance ωₖ and scalar measurement noise variance vₖ
ω = 0.3^2
v = 0.3^2

# initialize dynamics
Q = Matrix{Float64}(I, 2, 2)
R = v
dynamics = Model(f,h,u,Q,R)

Model(f, h, u, [1.0 0.0; 0.0 1.0], 0.09)

Define Bootstrap Particle Filter

In [65]:
Base.@kwdef mutable struct Particle_Filter
    model::Model
    TimeUpdate::Function
    MeasurementUpdate::Function
    Resampler::Function
    likelihoods::Vector
    particles::Array
end

function TimeUpdate(x, model, u, w)
    x_plus = Array{Float64}(undef, n, size(x,2))
    for i = axes(x,2)
        x_plus[:,i] = model.f(x[:,i],u,w[:,i])
    end
    return x_plus
end

function MeasurementUpdate!(particle_filter, model, y)
    x = particle_filter.particles
    measurement_error = y.-model.h(x,0)
    likelihoods = exp.((-1/2)*(measurement_error.^2)*inv(model.R))
    particle_filter.likelihoods = particle_filter.likelihoods.*likelihoods./(sum(particle_filter.likelihoods.*likelihoods))
end


function Resampler(particle_filter)
    x_resampled = fill(NaN, size(particle_filter.particles))
    CDF = cumsum(particle_filter.likelihoods)
    for i = 1:length(particle_filter.likelihoods)
        x_resampled[:,i] = particle_filter.particles[:,findfirst(CDF .>= rand(1))]
    end
    particle_filter.particles = x_resampled
    particle_filter.likelihoods = Vector(fill(1,(L)))
end

Resampler (generic function with 1 method)

Define some utility functions

In [67]:
# function: check_constraints
# input: x - 2D row vector of state particles
# output: 1 x length(x) vector of 1s and 0s, 1 being a state violation
function check_constraints(x)
    constraint_count = fill(0.0f0,size(x,2))
    for i = eachindex(constraint_count)
        in_region1 = (x1_lowerlim < x[1,i] < x1_upperlim) && (y1_lowerlim < x[2,i] < y1_upperlim)
        in_region2 = (x2_lowerlim < x[1,i] < x2_upperlim) && (y2_lowerlim < x[2,i] < y2_upperlim)

        if(in_region1||in_region2)
            constraint_count[i] = 1
        end
    end
    return constraint_count
end

####
# function: gpu_generate_Xi
# input: L = number of particles
# output: Ξ₀ = Array of randomly sampled states, size: [2 x L]
###
function gpu_generate_Xi(L :: Int64, n :: Int64, μ)
    # Gaussian Density with mean vector μ_x0 and covariance matrix Σ_x0
    μ_x0 = CuArray(μ)
    Σ_x0 = (0.5*I)

    # randomly sample initial states Ξ following Gaussian density
    Ξ₀ = CuArray{Float64}(undef,n,L)
    Ξ₀ = μ_x0.+sqrt(Σ_x0)*CUDA.randn(n,L)
    return (Ξ₀)
end

function gpu_sample_gaussian_distribution(mean, var, dims)
    w = CuArray{Float64}(undef, dims[1],dims[2],dims[3])
    w = mean.+sqrt(var)*CUDA.randn(dims[1],dims[2],dims[3])
    return w
end

gpu_sample_gaussian_distribution (generic function with 1 method)

In [68]:
# function: constraint_violation_kernel!
# objective: calculate constraint violation rates
function constraint_violation_kernel!(SSA_constraints,T,M,state,u, state_violation_count, i)
    # unpack SSA_constraints struct
    Ulim = SSA_constraints.Ulim
    x1_upperlim = SSA_constraints.x1_upperlim
    x1_lowerlim = SSA_constraints.x1_lowerlim
    y1_upperlim = SSA_constraints.y1_upperlim
    y1_lowerlim = SSA_constraints.y1_lowerlim
    x2_upperlim = SSA_constraints.x2_upperlim
    x2_lowerlim = SSA_constraints.x2_lowerlim
    y2_upperlim = SSA_constraints.y2_upperlim
    y2_lowerlim = SSA_constraints.y2_lowerlim

    index = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    stride = gridDim().x * blockDim().x

    # compare each trajectory with state constraints
    for j = index:stride:M
        for t = 1:T

            in_region1 = (x1_lowerlim < state[1,j,t] < x1_upperlim) && (y1_lowerlim < state[2,j,t] < y1_upperlim)
            in_region2 = (x2_lowerlim < state[1,j,t] < x2_upperlim) && (y2_lowerlim < state[2,j,t] < y2_upperlim)

            if(in_region1||in_region2)
                state_violation_count[j,t] = 1
            end
        end
    end


    return 
end

# function: launch_constraint_kernel!
# objective: launch kernel for violation rate calculation
function launch_constraint_kernel!(SSA_constraints,T,M,state,u, state_violation_count, i)
    kernel = @cuda launch=false constraint_violation_kernel!(SSA_constraints,T,M,state,u, state_violation_count, i)
    config = launch_configuration(kernel.fun)
    threads = min(M, config.threads)
    blocks = cld(M, threads)

    CUDA.@sync begin
        kernel(SSA_constraints,T,M,state,u, state_violation_count, i; threads, blocks)
    end
end

launch_constraint_kernel! (generic function with 1 method)

In [69]:
# function: xkprime!
# input:
# - SSA_params: struct of parameters containing L and N
# - u: array of control sequences [1 X L]
# - w: array of random noise [2 x M x N]
# - xkprime: batch of Monte Carlo sampled trajectories
# - dynamics: open-loop dynamics
function xkprime!(SSA_params, u, w, xkprime, dynamics)
    L = SSA_params.L
    N = SSA_params.N
    for i = 1:L
        for t = 1:N-1
            u[i,t] = dynamics.u(xkprime[:,i,t])
            xkprime[:,i,t+1] = dynamics.f(xkprime[:,i,t], u[i,t], w[:,i,t])
        end
    end
end


# function: xk2prime
# input:
# - N: time horizon
# - M: Monte Carlo sample number 
# - u: array of control sequences [1 X L]
# - w: array of random noise [2 x M x N]
# - xk2prime: batch of Monte Carlo sampled trajectories
function xk2prime!(N, M, u, w, xk2prime, i)
    for j = 1:M
        for t = 1:N-1
            xk2prime[:,j,t+1] = dynamics.f(xk2prime[:,j,t],u[i,t],w[:,j,t])
        end
    end
end


function J(x,u)
    return x'*x + u'*u
end

J (generic function with 1 method)

Define the State Selection Algorithm

In [70]:
# function: state_selection_algorithm
# inputs:
# - Ξ: particle density
# - SSA_params: struct of parameters
# - SSA_constraints: struct of state and input constraints
function state_selection_algorithm(J,Ξ,SSA_params,SSA_constraints)
    n = SSA_params.n
    L = SSA_params.L
    N = SSA_params.N
    M = SSA_params.M

    # intialize state array
    # state = CUDA.fill(1.0f0, (n,L,N))
    state = fill(1.0f0, (n,L,N))

    # fill state array with intial particle density
    state[:,:,1] = Ξ

    # initalize input array
    # u = CUDA.fill(0.0f0, L,N)
    u = fill(0.0f0, L,N)

    # generate random noise sequence Wprime for time horizon N for 
    # state density with num particles L
    # w = (gpu_sample_gaussian_distribution(0, ω, (n,L,N)))
    w = Array(gpu_sample_gaussian_distribution(0, ω, (n,L,N)))
    w2 = Array(gpu_sample_gaussian_distribution(0, ω, (n,L,N)))

    # ### First, lets generate the x' trajectories for time horizon N for each particle in state density Xi ###
    # CUDA.@sync launch_xprime_kernel!(state, N, w, u)
    xkprime!(SSA_params,u,w,state,dynamics)

    # declare vectors for x'' trajectories, cost, and constraint violation rate calculations
    cost = fill(0.0f0, L)
    sampled_costs = CUDA.fill(0.0f0, M,N)
    state_violation_count = CUDA.fill(0.0f0, M,N)
    sampled_state_violations = CUDA.fill(0.0f0,L,N)
    sampled_control_violations = CUDA.fill(0.0f0,L,N)
    total_state_violations = fill(false,L)
    total_control_violations = fill(false,L)
    state_2prime = CUDA.fill(0.0f0, (n,M,N))


    Ξ = CuArray(Ξ)
    # iterate through each particle in Ξ and run M monte carlo simulations for each particle 
    for i = 1:L
        CUDA.@sync begin
            # calculate x'' trajectories

            mc_sample_index = (rand(1:L, M))
            state_2prime[:,:,1] = Ξ[:,mc_sample_index]
            xk2prime!(N, M, Array(u), w2, Array(state_2prime), i)
            # launch_xk2prime_kernel!(SSA_params, state_2prime, u, w2, i)

            # calculate cost and state/control violation rates
            launch_constraint_kernel!(SSA_constraints, N, M , state_2prime, CuArray(u), state_violation_count, i)

            # sum the sampled cost to calculate the cost of each L particles
            cost[i] = M*sum(state[:,i,:].^2) + sum(state_2prime.^2)

            # sum the violation counts to make an [L x N] array, which contains the total violations of each trajectory
            sampled_state_violations[i,:] = sum(state_violation_count, dims=1)

            # indicate which particles satisfy state constraints
            total_state_violations[i] = all(sampled_state_violations[i,:]/M .< α)
            # total_control_violations[i] = all(sampled_control_violations[i,:]/M .< α)
        end
    end

    # mask for feasible states
    feasibility_mask = total_state_violations

    u = Array(u)

    
    if(sum(feasibility_mask)==0) # if there are no feasible states, the feasible set is empty and SSA cannot proceed
        println("Feasible set is empty!")
        cost_val,candidate_index = findmin(cost)
        return Array(Ξ)[:,candidate_index], u[candidate_index,1], candidate_index, feasibility_mask
    else # otherwise, find feasible state with minimum cost
        cost_val, candidate_index = findmin(cost[feasibility_mask])
        # println(cost_val,candidate_index, Array(Ξ)[:,candidate_index])
        return Array(Ξ)[:,candidate_index], u[candidate_index,1], candidate_index, feasibility_mask
    end

    return
end

state_selection_algorithm (generic function with 1 method)

Now we run the simulation

In [71]:
## function: run_simulation(T)
# input:
#       T - run simulation for T time steps
# objective: run the bootstrap particle filter in conjunction with the SSA/CM for T time steps
function run_simulation(T,RUN_SSA=true,RUN_CM=false)

    
    sim_data = fill(0.0f0, (n,L,T))
    violation_rate = fill(0.0f0,T)
    x_candidate = fill(0.0f0, (n,T))

    # initialize and store true state
    x_true = Array{Float64}(undef, n, T+1)
    x_true[:,1] = μ.+ sqrt(Σ)*randn(2)

    # generate state density Xi according to Gaussian parameters
    Ξ = gpu_generate_Xi(SSA_params.L, SSA_params.n,μ)

    # set intial particle density of the bootstrap particle filter
    pf.particles = Array(Ξ)

    # start simultion loop with T time steps
    for t = 1:T
        sim_data[:,:,t] = pf.particles

        if(RUN_SSA) # run the state selection algorithm for the particle density
            CUDA.@sync candidate_state, u_star, candidate_index, feasibility_mask = state_selection_algorithm(J,pf.particles,SSA_params,SSA_constraints)
            x_candidate[:,t] = pf.particles[:,candidate_index]
            if(isinf(u_star)||isnan(u_star))
                println("Feasible Set is Empty!!")
                break
            end
        elseif(RUN_CM) # choose the conditional mean as the state estimates
            x_candidate[:,t] = mean(pf.particles, dims = 2)
            candidate_state = x_candidate[:,t]
        else
            error("Please choose a state selection type to simulate")
        end

        # check how many particles violate state constraints
        violation_rate[t] = sum(check_constraints(pf.particles))/L
        println("Violation Rate: ", violation_rate[t])

        # controller based on selected_state
        u_star = dynamics.u(candidate_state)
    
        ### BOOTSTRAP PARTICLE FILTER UPDATE ###

        # generate random noise
        w = Array(gpu_sample_gaussian_distribution(0, ω, (n,L,1)))
        w_true = sqrt(ω)*randn(2)
    
        # propagate particle density
        pf.particles = pf.TimeUpdate(pf.particles, dynamics, u_star, w)
       
        # propagate true state
        x_true[:,t+1] = dynamics.f(x_true[:,t], u_star, w_true)
    
        # take measurement of true state
        y = dynamics.h(x_true[:,t+1], sqrt(v)*randn())
    
        # calculate likelihoods of states based on measurement
        pf.MeasurementUpdate(pf,dynamics,y)
    
        # resample with these new likelihoods
        pf.Resampler(pf)
    end

    return x_candidate, sim_data, violation_rate
end

run_simulation (generic function with 3 methods)

In [72]:
# State/control constraint violation rates
α = 0.15
ϵ = 0.30
δ = 0.01

# State/control constraints
Ulim = 3
x1_upperlim = 5
x1_lowerlim = 3
y1_upperlim = 2
y1_lowerlim = -4

x2_upperlim = 5
x2_lowerlim = -2
y2_upperlim = -4
y2_lowerlim = -7

# state density mean and variance
μ = [7.75;-7.75]
Σ = 0.5^2

# Density and sampling parameters
L = 1000 # number of particles
M = 300  # number of Monte Carlo samples
N = 6    # time horizon
T = 20   # simulation total time steps
n = 2    # dimension of the state

# intialize parameters
SSA_params = Params(M, N, L, n, T)

# initialize constraints
SSA_constraints = Constraints(Ulim, α, ϵ, δ, 
    x1_upperlim, x1_lowerlim, 
    y1_upperlim, y1_lowerlim,
    x2_upperlim, x2_lowerlim, 
    y2_upperlim, y2_lowerlim)

# initialize particle filter
likelihoods = Vector(fill(1,(L)))
pf = Particle_Filter(dynamics, TimeUpdate, MeasurementUpdate!, Resampler, likelihoods, Array{Float64}(undef))

Particle_Filter(Model(f, h, u, [1.0 0.0; 0.0 1.0], 0.09), TimeUpdate, MeasurementUpdate!, Resampler, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], fill(-2.240034580230713))

In [73]:
run_ssa = true
run_cm = false
x_candidate, sim_data, ssa_violation = run_simulation(T,run_ssa,run_cm);

Violation Rate: 0.0
Violation Rate: 0.0
Violation Rate: 0.0
Violation Rate: 0.005
Violation Rate: 0.041
Violation Rate: 0.034
Violation Rate: 0.0
Violation Rate: 0.0
Violation Rate: 0.0
Violation Rate: 0.0
Violation Rate: 0.0
Violation Rate: 0.0
Violation Rate: 0.0
Violation Rate: 0.0
Violation Rate: 0.0
Violation Rate: 0.0
Violation Rate: 0.0
Violation Rate: 0.0
Violation Rate: 0.0
Violation Rate: 0.0


Plot the results of the SSA particle filter

In [74]:
using Plots
using LaTeXStrings

In [75]:
function rectangle_from_coords(xb,yb,xt,yt)
    [
        xb  yb
        xt  yb
        xt  yt
        xb  yt
        xb  yb
        NaN NaN
    ]
end

function animate_frame(i,sim_data,x_candidate)
    xlims!(-15,15)
    ylims!(-17,7)
    # plot the particle density
    plot!(sim_data[1,:,i],sim_data[2,:,i],seriestype=:scatter,
    label=false,
    ms=MARKER_SIZE,
    z_order=:back)

    # plot the candidate state
    scatter!((x_candidate[1,i], x_candidate[2,i]),
    label = false,
    mc =:red,
    z_order=:front)

    # plot the state constraints
    plot!(state_constraints[:,1], state_constraints[:,2],
        label = false,
        lc=:black)

    xlabel!(L"z_k")
    ylabel!(L"h_k")
end

animate_frame (generic function with 1 method)

In [76]:
# store rectangle coordinatse
state_constraints = [
rectangle_from_coords(x1_lowerlim,y1_lowerlim,x1_upperlim,y1_upperlim)
rectangle_from_coords(x2_lowerlim,y2_lowerlim,x2_upperlim,y2_upperlim)]
MARKER_SIZE = 1.5
ANIMATE = true

true

In [77]:
# plot initial particle density
plot(sim_data[1,:,1],sim_data[2,:,1],seriestype=:scatter,label=false,ms=MARKER_SIZE)
scatter
scatter!((x_candidate[1,1], x_candidate[2,1]),
label = false,
mc =:red,
z_order=:front)

# make GIF
if(ANIMATE)
    anim = @animate for i = 1:T
        animate_frame(i,sim_data,x_candidate)
    end

    gif(anim, "Saved_plots/ssa.gif",fps=10)
end

# make plot
for i = 1:T
    animate_frame(i,sim_data,x_candidate)
end
savefig("Saved_plots/ssa_plot.png")

┌ Info: Saved animation to c:\Users\remba\UCSD\SAS Lab\SSA_Julia\Saved_plots\ssa.gif
└ @ Plots C:\Users\remba\.julia\packages\Plots\Ec1L1\src\animation.jl:156


"c:\\Users\\remba\\UCSD\\SAS Lab\\SSA_Julia\\Saved_plots\\ssa_plot.png"

Compare performance of a nominal particle filter using the conditional mean

In [78]:
run_ssa = false
run_cm = true
cm_x_candidate, cm_sim_data, cm_violation = run_simulation(T,run_ssa,run_cm);

Violation Rate: 0.0
Violation Rate: 0.0
Violation Rate: 0.031
Violation Rate: 0.199
Violation Rate: 0.15
Violation Rate: 0.373
Violation Rate: 0.0
Violation Rate: 0.0
Violation Rate: 0.0
Violation Rate: 0.0
Violation Rate: 0.0
Violation Rate: 0.0
Violation Rate: 0.0
Violation Rate: 0.0
Violation Rate: 0.0
Violation Rate: 0.0
Violation Rate: 0.0
Violation Rate: 0.0
Violation Rate: 0.0
Violation Rate: 0.0


In [79]:
# plot initial particle density
plot(cm_sim_data[1,:,1],cm_sim_data[2,:,1],seriestype=:scatter,label=false,ms=MARKER_SIZE)
scatter
scatter!((cm_x_candidate[1,1], cm_x_candidate[2,1]),
label = false,
mc =:red,
z_order=:front)

# make GIF
if(ANIMATE)
    anim = @animate for i = 1:T
        animate_frame(i,cm_sim_data,cm_x_candidate)
    end

    gif(anim, "Saved_plots/cm.gif",fps=10)
end

# make plot
for i = 1:T
    animate_frame(i,cm_sim_data,cm_x_candidate)
end
savefig("Saved_plots/cm_plot.png")


┌ Info: Saved animation to c:\Users\remba\UCSD\SAS Lab\SSA_Julia\Saved_plots\cm.gif
└ @ Plots C:\Users\remba\.julia\packages\Plots\Ec1L1\src\animation.jl:156


"c:\\Users\\remba\\UCSD\\SAS Lab\\SSA_Julia\\Saved_plots\\cm_plot.png"

Compare state violation rates

In [80]:
plot(ssa_violation, label = "State Selection Algorithm", shape =:utriangle, ms = 5)
plot!(cm_violation, label = "Conditional Mean", shape =:circle, ms = 4)
savefig("Saved_plots/violation_rates.png")

"c:\\Users\\remba\\UCSD\\SAS Lab\\SSA_Julia\\Saved_plots\\violation_rates.png"